# data_fetching.ipynb — Cricbuzz LiveStats

Notebook to fetch, parse and generate INSERT SQL for your local `cricketdb`. Fill the configuration cell before running cells that call the network or connect to DB.

In [ ]:
RAPIDAPI_KEY = '72d9f8fec5mshfc0f2f47123393ep150dedjsn98f5405906fe'
RAPIDAPI_HOST = 'cricbuzz-cricket.p.rapidapi.com'

DB_HOST = '127.0.0.1'
DB_PORT = 3306
DB_USER = 'root'
DB_PASS = ''
DB_NAME = 'cricketdb'
print('Update the variables and run cells as needed')

In [ ]:
import requests

def rapidapi_get(path, params=None):
    url = f'https://{RAPIDAPI_HOST}{path}'
    headers = {'x-rapidapi-key': RAPIDAPI_KEY, 'x-rapidapi-host': RAPIDAPI_HOST}
    r = requests.get(url, headers=headers, params=params, timeout=15)
    r.raise_for_status()
    return r.json()

# Example usage (commented):
# data = rapidapi_get('/matches/v1/live')
# from pprint import pprint
# pprint(data)
print('rapidapi_get defined')

In [ ]:
import json, datetime

def json_to_mysql_insert(table, rows, cols=None):
    if not rows:
        return ''
    if cols is None:
        cols = list(rows[0].keys())
    def fmt(v):
        if v is None:
            return 'NULL'
        if isinstance(v, (int, float)):
            return str(v)
        if isinstance(v, bool):
            return '1' if v else '0'
        s = str(v).replace('\\','\\\\').replace("'","\\'")
        return f"'{s}'"
    vals = []
    for r in rows:
        vals.append('(' + ','.join(fmt(r.get(c)) for c in cols) + ')')
    return f"INSERT INTO {table} ({','.join(cols)}) VALUES\n" + ',\n'.join(vals) + ';'

In [ ]:
teams_data = {"list": [{"teamId": 2, "teamName": "India", "teamSName": "IND", "imageId": 719031, "countryName": "India"}, {"teamId": 3, "teamName": "Pakistan", "teamSName": "PAK", "imageId": 591986, "countryName": "Pakistan"}]}
print('teams_data loaded, count=', len(teams_data.get('list',[])))

In [ ]:
rows = []
for it in teams_data.get('list',[]):
    rows.append({'team_id': it.get('teamId'), 'name': it.get('teamName'), 'country': it.get('countryName'), 'raw_json': json.dumps(it, ensure_ascii=False)})
sql = json_to_mysql_insert('teams', rows, cols=['team_id','name','country','raw_json'])
open('/mnt/data/teams_example_inserts.sql','w',encoding='utf8').write(sql)
print('Wrote /mnt/data/teams_example_inserts.sql')